<a href="https://colab.research.google.com/github/fersoler/LogiCoLab/blob/main/LogicaPropEquivSat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Equivalencias, formas normales y satisfacibilidad en lógica proposicional

Este cuaderno forma parte del [proyecto *LogiCoLab*](https://github.com/fersoler/LogiCoLab), publicado con licencia MIT.

👇 Para cargar las funciones que se utilizan a lo largo del cuaderno, pulsar sobre ▶ en el siguiente bloque y esperar a que aparezca ✅ a la izquierda. Si no ves el signo ▶, pasa el cursor por encima de "Mostrar código".

In [11]:
#@title
from IPython.display import display, HTML, Math, Latex
import graphviz
import pydot
import networkx as nx
import nltk
from nltk.grammar import FeatureGrammar, FeatDict
from nltk import ChartParser, Tree
import sys
from subprocess import *
import re
import sympy
from sympy import *
from sympy.logic import simplify_logic
from sympy.logic.boolalg import Implies, Equivalent, eliminate_implications
import numpy as np
import itertools
import pandas as pd
init_printing()


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

# Or = Symbol('Or')
# And = Symbol('And')
# Not = Symbol('Not')
# Implies = Symbol('Implies')
# Equivalent = Symbol('Equivalent')
p, q, r, s, t = symbols('p q r s t')

# Lists with all sonnectives and auxiliary symbols in the alphabet
connectives = ['<->', '->', '&', '|', '¬']
auxSymbs = ['(', ')']
atomsPQRST = ['p','q','r','s','t']

# Converts a true/false value to 1/0
def TF_2_10(val):
  if val:
    return 1
  else:
    return 0

# Function to get all atoms in a expression
def getAtoms(expr):
  out = expr
  for i in connectives+auxSymbs:
    out = out.replace(i,' ')
  out = set(out.split())
  return out

# Function to get all subformulas of a given sympy expression
def subforms(expr):
  s = list([expr])
  for arg in np.flip(expr.args):
    s = list(dict.fromkeys(subforms(arg)+s))
  return s

grammarText = """
% start F
F[p=atm, s=?s] -> '(' F[s=?s] ')'

F[p=not, s=[c=Not, l=?l]] -> '¬' F[p=not, s=?l]
F[p=not, s=[c=Not, l=?l]] -> '¬' F[p=atm, s=?l]

F[p=and, s=[c=And, l=?l, r=?r]] -> LAnd[s=?l] '&' LAnd[s=?r] | LAnd[s=?l] '&' F[p=and, s=?r]
LAnd[s=?s] -> F[p=not, s=?s] | F[p=atm, s=?s]

F[p=or, s=[c=Or, l=?l, r=?r]] -> LOr[s=?l] '|' LOr[s=?r] | LOr[s=?l] '|' F[p=or, s=?r]
LOr[s=?s] -> F[p=and, s=?s] | F[p=not, s=?s] | F[p=atm, s=?s]

F[p=imp, s=[c=Implies, l=?l, r=?r]] -> LImp[s=?l] '->' LImp[s=?r] | LImp[s=?l] '->' F[p=imp, s=?r]
LImp[s=?s] -> F[p=or, s=?s] | F[p=and, s=?s] | F[p=not, s=?s] | F[p=atm, s=?s]

F[p=eqv, s=[c=Equivalent, l=?l, r=?r]] -> LEqv[s=?l] '<->' LEqv[s=?r] | LEqv[s=?l] '<->' F[p=eqv, s=?r]
LEqv[s=?s] -> F[p=imp, s=?s] | F[p=or, s=?s] | F[p=and, s=?s] | F[p=not, s=?s] | F[p=atm, s=?s]

F[p=atm, s=[c=Symbol, l=p]] -> 'p'
F[p=atm, s=[c=Symbol, l=q]] -> 'q'
F[p=atm, s=[c=Symbol, l=r]] -> 'r'
F[p=atm, s=[c=Symbol, l=s]] -> 's'
F[p=atm, s=[c=Symbol, l=t]] -> 't'
"""
# Original grammar and parser
grammar = FeatureGrammar.fromstring(grammarText)
parser = nltk.parse.FeatureChartParser(grammar)
# Test:
# pfml = parser.parse('¬ p & q'.split())
# fml = next(pfml)
# struct = fml.label()
# print(struct['s'])
# [ c = 'And'                    ]
# [                              ]
# [     [ c = 'Not'            ] ]
# [ l = [                      ] ]
# [     [ l = [ c = 'Symbol' ] ] ]
# [     [     [ l = 'p'      ] ] ]
# [                              ]
# [ r = [ c = 'Symbol' ]         ]
# [     [ l = 'q'      ]         ]

# The previous grammar only parses formulas with p, q, r, s and t. To extend the
# set of accepted variables, this function adds the rules for all propositions
# in 'listSymbs'. Test: print(grammarWithSymbols(["gato", "perro"]))
def grammarWithSymbols(listSymbs):
  newLines = ""
  for s in listSymbs:
    newLines = newLines + "\nF[p=atm, s=[c=Symbol, l="+s+"]] -> '"+s+"'"
  return grammarText+newLines

# Function to parse a formula that may contain propositons different to p,...,t.
# It returns the feature structure corresponding to 's'
def parseFml(fml):
  atoms = getAtoms(fml)
  toParse = fml
  # To avoid confusion of -> and <->
  toParse = toParse.replace('<->', ' = ')
  for c in connectives+auxSymbs:
    toParse = toParse.replace(c,' '+c+' ')
  # Back to <->
  toParse = toParse.replace('=','<->')
  toParse = toParse.split()
  if(atoms.issubset(set(atomsPQRST))):
    pfml = parser.parse(toParse)
  else:
    extGramm = grammarWithSymbols(atoms.difference(set(atomsPQRST)))
    lfgExt = FeatureGrammar.fromstring(extGramm)
    expPars = nltk.parse.FeatureChartParser(lfgExt)
    pfml = expPars.parse(toParse)
  fml = next(pfml)
  struct = fml.label()
  return struct['s']
# Test:
# print(parseFml('gato -> felino'))
# [ c = 'Implies'        ]
# [                      ]
# [ l = [ c = 'Symbol' ] ]
# [     [ l = 'gato'   ] ]
# [                      ]
# [ r = [ c = 'Symbol' ] ]
# [     [ l = 'felino' ] ]

def nltk_to_sympy(featStruct):
  sym_ops = {
      'Implies': sympy.Implies,
      'Or': sympy.Or,
      'And': sympy.And,
      'Not': sympy.Not,
      'Equivalent': sympy.Equivalent
      }
  if featStruct['c'] == 'Symbol':
    return Symbol(featStruct['l'])
  if featStruct['c'] == 'Not':
    left = nltk_to_sympy(featStruct['l'])
    return sympy.Not(left)
  # Binary connectives:
  left  = nltk_to_sympy(featStruct['l'])
  right = nltk_to_sympy(featStruct['r'])
  op = featStruct['c']
  return sym_ops[op](left, right)
# Test:
# fs = parseFml('¬(¬p & q <-> r)')
# symexp = nltk_to_sympy(fs)
# symexp

# Function to convert a list of formulas into their
# conjunction
def ListToAnd(listFmls):
  if len(listFmls) == 0:
    return True
  if len(listFmls) == 1:
    return listFmls[0]
  else:
    return And(listFmls[0], ListToAnd(listFmls[1:]))


# Function to build a tree with the evaluation of a formula in a given
# interpretation
def evalTree(G, fml, val, nodeN, parN = 0):
  col = 'black'
  if fml.subs(val) == True:
    col = 'green'
  if fml.subs(val) == False:
    col = 'red'
  G.add_node(nodeN, label=printTXT(fml), shape='rect', color=col)
  if(parN != 0):
    G.add_edge(parN, nodeN, dir='back')
  fmlArgs = fml.args
  for i in range(len(fmlArgs)):
    evalTree(G, fmlArgs[i], val, str(nodeN)+str(i), nodeN)


def printFml(sympyFml, conn):
  c = sympyFml.func
  a = sympyFml.args
  if c == Symbol:
    return str(sympyFml)
  if c == Not:
    c2 = a[0].func
    if (c2 == Symbol) | (c2 == Not):
      s = conn[0] + printFml(a[0], conn)
    else:
      s = conn[0] + "(" + printFml(a[0], conn) + ")"
    return s
  if c == And:
    fl = printFml(a[0], conn)
    opl = a[0].func
    if (opl == Symbol) | (opl == Not) | (opl == And):
      strl = fl
    else:
      strl = "("+fl+")"
    for i in range(1,len(a)):
      opr = a[i].func
      fr = printFml(a[i], conn)
      if (opr == Symbol) | (opr == Not) | (opr == And):
        strl = strl + conn[1] +fr
      else:
        strl = strl + conn[1] +"("+fr+")"
    return strl
  if c == Or:
    fl = printFml(a[0], conn)
    opl = a[0].func
    if (opl == Symbol) | (opl == Not) | (opl == Or) | (opl == And):
      strl = fl
    else:
      strl = "("+fl+")"
    for i in range(1,len(a)):
      opr = a[i].func
      fr = printFml(a[i], conn)
      if (opr == Symbol) | (opr == Not) | (opl == Or):  # | (opr == And)
        strl = strl + conn[2] +fr
      else:
        strl = strl + conn[2] +"("+fr+")"
    return strl
  if c == Implies:
    fl = printFml(a[0], conn)
    fr = printFml(a[1], conn)
    opl = a[0].func
    opr = a[1].func
    if (opl == Symbol) | (opl == Not) | (opl == And) | (opl == Or):
      strl = fl
    else:
      strl = "("+fl+")"
    if (opr == Symbol) | (opr == Not) | (opr == And) | (opl == Or):
      strr = fr
    else:
      strr = "("+fr+")"
    return strl+conn[3]+strr
  if c == Equivalent:
    fl = printFml(a[0], conn)
    fr = printFml(a[1], conn)
    opl = a[0].func
    opr = a[1].func
    if (opl == Symbol) | (opl == Not) | (opl == And) | (opl == Or):
      strl = fl
    else:
      strl = "("+fl+")"
    if (opr == Symbol) | (opr == Not) | (opr == And) | (opl == Or):
      strr = fr
    else:
      strr = "("+fr+")"
    return strl+conn[4]+strr

def printTXT(sympyFml):
  return printFml(sympyFml, ['¬', ' ∧ ', ' ∨ ', ' → ', ' ↔ '])


def printLaTeX(sympyFml):
  return printFml(sympyFml, ['\\lnot ', '\\land ', '\\lor ', '\\to ', '\\leftrightarrow '])

## Equivalencias y esquemas de equivalencia

Recodemos que dos fórmulas $A$ y $B$ son *equivalentes* (${}A\equiv{}B$) cuando en cualquier interpretación ${}v $ su valor de verdad es idéntico, es decir, ${}v (A) = {}v (B)$.

✏ **Ejercicio.** Justificar por qué las tres siguientes afirmaciones son equivalentes:
- $A\equiv B$.
- $\{A\}\models B$ y $\{B\}\models A$.
- $\models A \leftrightarrow B$.

### Esquemas de equivalencia de la lógica proposicional

Una expresión como:
$${}A\to{}B \equiv \lnot{}A\lor{}B$$
es un *esquema de equivalencia*, lo que significa que si sustituimos ${}A$ y ${}B$ por cualesquiera fórmulas de $\mathcal{L}_{LP}$, obtenemos una equivalencia.

Existen infinitos esquemas de equivalencia, de los cuales algunos son especialmente relevantes y por ello tienen nombre propio. En la siguiente tabla usamos $\top$ para representar cualquier tautología y $\bot$ cualquier contradicción:


<center>
<table>
<tr><td>

| **Nombre** | **Expresión**                                 |
|:-----------|:----------------------------------------------|
| Conmutativa|  ${}A\lor{}B$ $\equiv$ ${}B\lor{}A$ |
|            |${}A\land{}B$ $\equiv$ ${}B\land{}A$ |
| Asociativa|  ${}A$ $\lor$ $({}B\lor{}C)$ $\equiv$ $({}A\lor{}B)$ $\lor$ ${}C$ |
|            |  ${}A$ $\land$ $({}B\land{}C)$ $\equiv$ $({}A\land{}B)$ $\land$ ${}C$ |
| Distributiva | ${}A$ $\lor$ $({}B\land{}C)$ $\equiv$ $({}A\lor{}B)$ $\land$ $({}A\lor{}C)$ |
| | ${}A$ $\land$ $({}B\lor{}C)$ $\equiv$ $({}A\land{}B)$ $\lor$ $({}A\land{}C)$ |
| Doble negación | $\lnot\lnot{}A$ $\equiv$ ${}A$ |
| De Morgan | $\lnot$$({}A\lor{}B)$ $\equiv$ $\lnot{}A$ $\land$ $\lnot{}B$ |
| | $\lnot$$({}A\land{}B)$ $\equiv$ $\lnot{}A$ $\lor$ $\lnot{}B$ |
| Idempotencia | ${}A\lor{}A$ $\equiv$ ${}A$ |
|  | ${}A\land{}A$ $\equiv$ ${}A$ |
| Absorción | ${}A$ $\lor$ $({}A\land{}B)$ $\equiv$ ${}A$ |
| | ${}A$ $\land$ $({}A\lor{}B)$ $\equiv$ ${}A$ |
| Tautología | ${}A\lor \top$ $\equiv$ $\top$ |
| | ${}A\land \top$ $\equiv$ ${}A$ |
| | ${}A\lor \lnot {}A$ $\equiv$ $\top$ |
| Contradicción | ${}A\lor \bot$ $\equiv$ ${}A$ |
| | ${}A\land \bot$ $\equiv$ $\bot$ |
| | ${}A\land \lnot {}A$ $\equiv$ $\bot$ |
| Condicional | ${}A$$\to$${}B$ $\equiv$ $\lnot{}A$$\lor$${}B$
| | $\lnot$$({}A$$\to$${}B)$ $\equiv$ ${}A$ $\land$ $\lnot{}B$
| Bicondicional | ${}A↔{}B$ $\equiv$ $({}A\to{}B)$ $\land$ $({}B\to{}A)$
| | $\lnot$ $({}A↔{}B)$ $\equiv$ $({}A\lor{}B)$ $\land$ $(\lnot{}A$$\lor$$\lnot{}B)$

</td></tr> </table>
</center>

✏ **Ejercicio.** Usar tablas de verdad para verificar las equivalencias de la tabla anterior.

Un uso importante de las equivalencias es que nos permiten transformar las fórmulas de $\mathcal{L}_{LP}$ en ciertas *formas normales* que permiten estudiar fácilmente cuestiones como la satisfacibilidad o validez de fórmulas proposicionales. Lo hacemos usando el siguiente resultado.


**Teorema de intercambio**. Si ${}A\equiv{}B$, dada una fórmula $C$ que tiene ${}A$ como subfórmula, el resultado de sustituir  ${}A$ por ${}B$ en $C$, que denotamos como $C({}A/{}B)$, es equivalente a $C$,
$${}A\equiv{}B \quad \Rightarrow \quad C \equiv C({}A/{}B).$$
En el caso de que ${}A$ sea la propia $C$, tenemos que $C({}A/{}B) = {}B$.

**Ejemplo.** La fórmula $p \to ($<font color="red">$q\to r$</font>$)$ contiene <font color="red">$q \to r$</font> como subfórmula. Dado que <font color="red">$q\to r$</font> $\equiv$ <font color="blue">$\lnot q\lor r$</font> (una de las equivalencias del condicional), tenemos que
<center>
$p \to ($<font color="red">$q\to r$</font>$)$ $\equiv$ $p \to ($<font color="blue">$\lnot q\lor r$</font>$)$.
</center>

In [12]:
#@title ### Sustitución de subfórmulas usando el **teorema de intercambio**

#@markdown Introducir una fórmula para aplicar el teorema de intercambio y dos listas **A** y
#@markdown **B** de fórmulas separadas por comas. Cada fórmula de la lista **A** debe ser equivalente
#@markdown a la que ocupa la misma posición en **B** para que se pueda aplicar el intercambio.

#@markdown Probar con los valores de ejemplo, que se pueden modificar. Al pulsar sobre ▶
#@markdown se irán aplicando las sustituciones una a una.

#@markdown Usar "¬" para la negación, "&" para la conjunción, "|" para la
#@markdown disyunción, "->" para el condicional y "<->" para el bicondicional.

Formula = "p -> (p & r)" #@param ["p -> ¬(q -> r)"] {allow-input: true}
listaA  = "¬(q -> r), p -> (q & ¬r), ¬p | (q & ¬r)" #@param ["¬(q -> r), p -> (q & ¬r), ¬p | (q & ¬r)"] {allow-input: true}
listaB  = "q & ¬r, ¬p | (q & ¬r), (¬p | q) & (¬p | ¬r)" #@param ["q & ¬r, ¬p | (q & ¬r), (¬p | q) & (¬p | ¬r)"] {allow-input: true}

fml = nltk_to_sympy(parseFml(Formula))
list1 = [nltk_to_sympy(parseFml(f)) for f in listaA.split(',')]
list2 = [nltk_to_sympy(parseFml(f)) for f in listaB.split(',')]

if(len(list1) != len(list2)):
  print("Las listas A y B deben tener el mismo número de fórmulas.")
else:
  Math("\\text{Fórmula original: }\\quad " + printLaTeX(fml))
  theFml = fml
  for i in range(len(list1)):
    if(simplify_logic(Equivalent(list1[i], list2[i])) == True):
      print("")
      Math("\\text{Sustitución "+ str(i+1) +": }\\quad " +
           printLaTeX(list1[i])+"  \\equiv "+printLaTeX(list2[i]))
      theFml = theFml.subs({list1[i]: list2[i]})
      Math("\\text{Resultado: }\\quad\\quad "+
           printLaTeX(theFml))
    else:
      print("")
      Math("\\text{Sustitución "+ str(i+1) +": }\\quad " +
           printLaTeX(list1[i])+"  \\not\\equiv "+printLaTeX(list2[i])
           +"\\quad \\text{(sustitución no aplicada)}")

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Formas normales

Usando el *teorema de intercambio* podemos transformar cualquier fórmulas de $\mathcal{L}_{LP}$ en una fórmula equivalente con una estructura sintáctica normalizada. Son las formas normales **conjuntiva** y **disyuntiva**.

**Definiciones**.
- Llamamos *literales* a las variables proposicionales o sus negaciones. Las fórmulas $p$, $\lnot q$, $\lnot r$, $s$, etc., son ejemplos de literales. Llamamos *complementarios* a cada par de literales en que uno de los dos es la negación del otro. Así, $p$ y $\lnot p$ son literales complementarios. En ocasiones usamos el símbolo $\bar{{}l }$ para referirnos al literal complementario de ${}l $.
- Una *conjunción elemental* es una conjunción de literales. Son conjunciones elementales $p \land \lnot q \land r$, $s \land \lnot s$, etc.
- Una *disyunción elemental* es una disyunción de literales. Son disyunciones elementales $\lnot p \lor q \lor r$, $p \lor \lnot r \lor \lnot p$, etc.
- Una fórmula está en *forma normal conjuntiva* (**FNC**) cuando se trata de una conjunción de disyunciones elementales. Su estructura es:
$$
({}l ^1_1 \lor \ldots \lor {}l ^1_{n_1})
\land ({}l ^2_1 \lor \ldots \lor {}l ^2_{n_2})
\land \ldots
\land ({}l ^m_1 \lor \ldots \lor {}l ^m_{n_m})
$$
- Una fórmula está en *forma normal disyuntiva* (**FND**) cuando se trata de una disyunción de conjunciones elementales. Su estructura es:
$$
({}l ^1_1 \land \ldots \land {}l ^1_{n_1})
\lor ({}l ^2_1 \land \ldots \land {}l ^2_{n_2})
\lor \ldots
\lor ({}l ^m_1 \land \ldots \land {}l ^m_{n_m})
$$

Las conjunciones y disyunciones elementales pueden tener varios literales, uno solo o incluso ser vacías. Una conjunción vacía es equivalente a $\top$ y una disyunción vacía a $\bot$.

Podemos transformar cualquier fórmula a su forma normal conjuntiva o disyuntiva usando el teorema de intercambio a través de los siguientes pasos:
1. Eliminamos los condicionales $\to$ y bicondicionales $↔$ usando las equivalencias de *Condicional* y *Bicondicional* (tabla de arriba). Tras este paso, llegamos a una fórmula que solo tiene negaciones, conjunciones y disyunciones.
2. Usando las equivalencias de *Doble negación* y *De Morgan* (simplificando cuando se pueda con *Tautología*, *Contradicción*, *Absorción* y *Idempotencia*), obtenemos la **forma normal negativa** en que las negaciones afectan solo a variables proposicionales.
3. Aplicamos las propiedad distributiva:
  - Para obtener la **forma normal conjuntiva (FNC)** aplicamos la equivalencia
  $${}A \lor ({}B\land{}C) \equiv ({}A\lor{}B) \land ({}A\lor{}C)$$
  hasta que ninguna conjunción se encuentre bajo el alcance de ninguna disyunción.
  - Para obtener la **forma normal disyuntiva (FND)** aplicamos la equivalencia
  $${}A \land ({}B\lor{}C) \equiv ({}A\land{}B) \lor({}A\land{}C)$$
  hasta que ninguna disyunción se encuentre bajo el alcance de ninguna conjunción.

  En ambos casos simplificamos como en el paso 2.

In [14]:
#@title ### Obtención de **formas normales conjuntiva y disyuntiva**

#@markdown Introducir una fórmula para aplicar el teorema de intercambio hasta llegar a sus formas
#@markdown normales conjuntiva y disyuntiva. Se muestra el resultado de cada uno de los tres
#@markdown pasos explicados arriba.

#@markdown Probar con los valores de ejemplo, que se pueden modificar, y pulsar sobre ▶.
#@markdown Debe tenerse en cuenta que usamos la librería `sympy` que puede realizar algunas
#@markdown simplificaciones y reordenaciones de subfórmulas.

#@markdown Usar "¬" para la negación, "&" para la conjunción, "|" para la
#@markdown disyunción, "->" para el condicional y "<->" para el bicondicional.


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'


Formula = "(¬q & (p -> q)) -> ¬p" #@param ["p -> ¬(q <-> r)", "(¬q & (p -> q)) -> ¬p"] {allow-input: true}

fml = nltk_to_sympy(parseFml(Formula))

print("Fórmula original:")
Math(printLaTeX(fml))

print("\n1. Eliminación de condicionales y bicondicionales:")
eliminate_implications(fml)

print("\n2. Forma normal negativa:")
to_nnf(fml)

print("\n3a. Forma normal conjuntiva:")
to_cnf(fml)
print("Simplificando trivialidades:")
simplify_logic(fml, form='cnf')

print("\n3b. Forma normal disyuntiva:")
to_dnf(fml)
print("Simplificando trivialidades:")
simplify_logic(fml, form='dnf')


Fórmula original:


<IPython.core.display.Math object>


1. Eliminación de condicionales y bicondicionales:


q ∨ (p ∧ ¬q) ∨ ¬p


2. Forma normal negativa:


...

True


3b. Forma normal disyuntiva:


q ∨ (p ∧ ¬q) ∨ ¬p

Simplificando trivialidades:


True

## Introducción al algoritmo DPLL para comprobación de satisfacibilidad

El [problema SAT](https://es.wikipedia.org/wiki/Problema_de_satisfacibilidad_booleana) consiste en determinar si una fórmula dada de la lógica proposicional es satisfacible. Es un conocido problema en ciencias de la computación por ser [NP-completo](https://es.wikipedia.org/wiki/NP-completo).

> 👉 Otras importantes nociones semánticas como la validez, equivalencia o consecuencia lógica se pueden `reducir` a la satisfacibilidad, ¿cómo?

Uno de los algoritmos más eficientes para resolver el problema SAT es el conocido como [DPLL](https://es.wikipedia.org/wiki/Algoritmo_DPLL), que debe su nombre a [M. Davis](https://es.wikipedia.org/wiki/Martin_Davis), [H. Putnam](https://es.wikipedia.org/wiki/Hilary_Putnam), [G. Logemann](https://en.wikipedia.org/wiki/George_Logemann) y [D.W. Loveland](https://en.wikipedia.org/wiki/Donald_W._Loveland).

El algoritmo parte de una fórmula en forma normal conjuntiva y trata de obtener una interpretación que la satisfaga. Para que una interepretación ${}v $ satisfaga una fórmula en FNC debe satisfacer cada una de sus disyunciones elementales. Observemos que si consideramos un literal ${}l $ verdadero (${}v ({}l ) = 1$):
- Cualquier disyunción elemental que contenga ${}l $ pasa a ser verdadera, por lo que ya hemos cumplido el objetivo de que quede satisfecha por ${}v $.
- Cualquier disyunción elemental que contenga $\bar{{}l }$ (literal complementario de ${}l $) solo será satisfecha por ${}v $ si alguno de los demás literales resulta verdadero, por lo que el objetivo de que tal disyunción elemental quede satisfecha por ${}v $ se puede reducir limitándolo al resto de literales.

El algoritmo DPLL trata de construir ${}v $ siguiendo los siguientes criterios:
- *Propagación unitaria*. Si una cláusula unitaria (tiene un solo literal al que aún no se ha asignado un valor de verdad), sólo puede satisfacerse haciendo que dicho literal sea verdadero. La propagación unitaria elimina todas las cláusulas que contienen un literal ${}l $ de una cláusula unitaria y descarta $\bar{{}l }$ de todas las cláusulas que contienen dicho complemento.
- *Eliminación de literales puros*. Si aparecen ocurrencias de un cierto literal ${}l $ en la fórmula pero no ocurre $\bar{{}l }$, a ${}l $ se le llama *literal puro*. Se puede tomar ${}v ({}l )=1$ y todas las cláusulas que contengan ${}l $ pasan a ser verdaderas.

A medida que se van asignando valores de verdad a literales, se van reduciendo tanto el número de cláusulas que quedan por satisfacerse como los literales de las cláusulas aún no satisfechas. Si todas las cláusulas son satisfechas, la fórmula resulta satisfacible, la interpretación ${}v $ que se ha construido la satisface. Si alguna cláusula no satisfecha se queda sin literales (recordemos que una disyunción vacía equivale a $\bot$), la búsqueda falla y se usa *backtracking* para elegir otra interpretación. Si no quedan interpretaciones posibles que probar, la fórmula no es satisfacible.

In [15]:
#@title ### Ejecución del algoritmo DPLL

#@markdown La fórmula introducida será convertida a FNC y se aplicará
#@markdown el algoritmo DPLL con las asignaciones que se vayan realizando.
#@markdown Presionar ▶ cada vez que se incluye una nueva asignación para
#@markdown ejecutar un nuevo paso del algoritmo.

#@markdown Las asignaciones se introducen como una lista separada por comas.
#@markdown Por ejemplo, "p: 1, q: 0" indica que $p$ toma el valor $1$ y $q$
#@markdown el valor $0$. Si la asignación introducida hace verdadera la
#@markdown formula, el algoritmo devuelve `True` o `1`.

#@markdown Cuando la interpretación que se va construyendo no satisface la
#@markdown formula, se puede hacer *backtracking* modificando las
#@markdown últimas decisiones.


#@markdown Usar "¬" para la negación, "&" para la conjunción, "|" para la
#@markdown disyunción, "->" para el condicional y "<->" para el bicondicional.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

Formula = "(¬q & (p -> q)) -> p" #@param ["p -> ¬(q <-> r)", "(¬q & (p -> q)) -> p"] {allow-input: true}
Interpretacion = "p: 0, q: 1" #@param ["p: 0, q: 1", "p: 1, q: 0"] {allow-input: true}

fml = nltk_to_sympy(parseFml(Formula))
if (Interpretacion.strip() == ''):
  valuation = []
else:
  valuation = [f.split(':') for f in Interpretacion.replace(' ','').split(',')]

print("Fórmula introducida:")
Math(printLaTeX(fml))

print("\nForma normal conjuntiva:")
fml1 = simplify_logic(fml, form='cnf')
fml1

for i in range(len(valuation)):
  print("")
  Math("\\text{Asumimos }v ("+
       str(valuation[i][0])+") = "+str(valuation[i][1]))
  fml1.subs(valuation[:i+1])


Fórmula introducida:


<IPython.core.display.Math object>


Forma normal conjuntiva:


p ∨ q

<IPython.core.display.Math object>

q

<IPython.core.display.Math object>

1

## Trabajando con `sympy`

En el bloque de código de más abajo tenemos la posibilidad de trabajar directamente con `sympy` librería de `Python` que usamos en este cuaderno para manipular fórmulas de lógica proposicional. Se puede encontrar documentación de `sympy` en la [página del proyecto](https://docs.sympy.org/latest/modules/logic.html).

In [16]:
# Escribe una fórmula:
formula = "(p -> q | r) & ¬q"

# Esto analiza sintácticamente la fórmula y la convierte
# a una expresión de sympy:
fml = nltk_to_sympy(parseFml(formula))

# Imprime la fórmula:
Math(printLaTeX(fml))
# Comprueba si es satisfacible y, si lo es, obtiene una interpretación:
print("\nSatisfacibilidad:")
satisfiable(fml)

<IPython.core.display.Math object>


Satisfacibilidad:


{r: True, q: False, p: False}

## Aplicación: resolución de sudokus mediante satisfacibilidad proposicional

El problema de la satisfacibilidad proposicional es [objeto de estudio](http://www.satisfiability.org/) tanto por su interés teórico (problema NP-completo) como por sus numerosas aplicaciones.

Computacionalmente, es un problema intratable, dado que la resolución de un problema con $n$ variables, puede requerir, en el caso peor, del orden de $2^n$ pasos de computación.

A modo de ejemplo, vamos a utilizar satisfacibilidad proposicional para resolver sudokus de tamaño $4\times 4$. Para orientarlo como un problema de satisfacibilidad proposicional, definimos, para cada una de las 16 celdas, cuatro variables proposicionales, que indican cada uno de los valores que puede tener la celda. Así `p142` significa que la celda correspondiente a la fila 1 columna 4 tiene un 2. El total de variables que necesitamos es $16\times 4 = 64$, por lo que una tabla de verdad tendría $2^{64}$ filas, del orden de $10^{20}$ (más de 100 veces la edad del universo en segundos). Podemos hacernos idea del coste computacional del problema y la pertinencia de disponer de algoritmos eficientes para resolver el problema SAT.

Representamos mediante fórmulas proposicionales la información necesaria para resolver un determinado sudoku, lo que incluye:
- Información general propia del *problema sudoku*, como que un mismo número no puede repetirse en una misma fila, columna o región, o que cada celda tiene un valor entre 1 y 4. Una fórmula como $p142 \to \lnot p132$ (que escribiremos como $\lnot p142 \lor \lnot p132$, para trabajar en FNC) indica que si hay un 2 en la fila 1 (columna 4) no puede haber otro 2 en la fila 1 (columna 3). Será necesario un gran número de fórmulas de este tipo.
- Información específica del sudoku que estamos resolviendo. Serán las fórmulas que definen las casillas que aparecen rellenas en el enunciado del problema.

Ofrecemos el código más abajo. Se puede introducir un problema modificando la cadena que se asigna a la variable `problema`. Luego pulsar ▶ para resolverlo, aparecerá impresa la tabla completa.

En los comentarios del código se puede ver cómo se van construyendo las fórmulas. El paso principal es el uso de `satisfiable` para buscar la interpretación (sudoku resuelto) que satisface la fórmula que contiene todas las restricciones del problema. En ciertos momentos, previamente, se utiliza `simplify_logic` para simplificar la fórmula.

El programa no es especialmente eficiente sobre Google Colab. Existen [otras implementaciones](https://github.com/liloheinrich/Sudoku) en `Python` mucho más eficientes.

In [17]:
# Definimos el problema rellenando los datos en la tabla del sudoku
problema = """
+---+---+---+---+
|   |   |   | 1 |
+---+---+---+---+
| 1 | 2 |   |   |
+---+---+---+---+
|   | 4 |   |   |
+---+---+---+---+
|   | 1 | 3 |   |
+---+---+---+---+
"""

# Lista de números que ocupan cada posición de la tabla
numbers = problema.replace('+','').replace('-','').replace('\n\n|','').replace('|\n\n','').replace(' ','').split('|')


datos = []  # Aquí vamos a ir guardando los literales que definen el problema
vals = [1,2,3,4]
for r in vals:
  for c in vals:
    dato = numbers[4*(r-1)+c-1]
    if(dato != ''):   # Si en la fila 'r' columna 'c' hay un dato
      datos.append('p'+str(r)+str(c)+dato) # Lo añadimos

# Función para definir una variable proposicional que indica que en la fila
# 'r' columna 'c' está el número 'n'
def prop(r,c,n):
  return Symbol("p"+str(r)+str(c)+str(n))

# Esta es la fórmula que vamos a ir construyendo con los datos del sudoku
# y las restricciones generales.
listaFml = True

# Comenzamos añadiendo los datos de entrada del problema específico
for d in datos:
  listaFml = Symbol(d) & listaFml

# RESTRICCIONES DEL SUDOKU DE 4x4:

# Cada celda contiene un solo valor entre 1 y 4:
for row in vals:
  for col in vals:
    options = False # Para construir los 4 posibles valores
    for n in vals:
      options = options | prop(row,col,n)
    # Lo añadimos a la lista de fórmulas y simplificamos:
    listaFml = simplify_logic(options & listaFml, form='cnf')
# Un número no puede estar repetido en la misma fila o columna
for row in vals:
  for col in vals:
    for n in vals:
      # Caso de que 'n' esté en (row, col):
      pr = prop(row,col,n)
      for val2 in vals:
        if row < val2:
          # Lo puede estar en otra posición (posterior) de la misma columna:
          listaFml = (~pr | ~prop(val2,col,n)) & listaFml
        if col < val2:
          # Ni de la misma columna:
          listaFml = (~pr | ~prop(row,val2,n)) & listaFml
    # Vamos simplificando la (gran) fórmula que se va construyendo:
    listaFml = simplify_logic(listaFml, form='cnf')
# En un mismo cuadro de 2x2 no puede ocurrir ningún número repetido:
for sR in [0,2]:    # Desplazamiento de fila (para recorrer los 4 cuadros 2x2)
  for sC in [0,2]:  # Desplazamiento de columna
    valsR = [1+sR,2+sR] # Posiciones de las filas en el cuadro actual
    valsC = [1+sC,2+sC] # Posiciones de las columnas en el cuadro actual
    for row in valsR:
      for col in valsC:
        for n in vals:
          # Si estuviera el número 'n' en la posición (row, col):
          pr = prop(row,col,n)
          for row2 in valsR:
            for col2 in valsC:
              # Para cada celda del cuadro 2x2 que no comparta fila ni columna
              # (pues ya estaban en restricciones anteriores):
              if (row != row2) & (col < col2):
                # El número 'n' no puede estar en dicha celda:
                listaFml = (~pr | ~prop(row2, col2, n)) & listaFml
    listaFml = simplify_logic(listaFml, form='cnf')

# Obtenemos la interpretación que satisface la fórmula que representa el sudoku:
solution = satisfiable(listaFml)

# Tomamos solo las variables verdaderas (número que está en cada celda):
verdaderas = [str(clave) for clave, valor in solution.items() if valor]
# Ordenamos (orden lexicográfico, para que siga el orden de las filas
# y columnas):
verdaderas.sort()

print("\n¡¡¡Sudoku resuelto!!!\n\n")

# Imprimimos la solución del sudoku:
print("+---+---+---+---+")
for i in range(4):
  for j in range(4):
    print('| '+verdaderas[i*4+j][-1], end=' ')
  print("|")
  print("+---+---+---+---+")



¡¡¡Sudoku resuelto!!!


+---+---+---+---+
| 4 | 3 | 2 | 1 |
+---+---+---+---+
| 1 | 2 | 4 | 3 |
+---+---+---+---+
| 3 | 4 | 1 | 2 |
+---+---+---+---+
| 2 | 1 | 3 | 4 |
+---+---+---+---+
